<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import warnings

warnings.filterwarnings('ignore')

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
query_3_1 = f'''
SELECT COUNT(id) cnt_vacancies
    FROM public.vacancies           
'''
vacancies_df = pd.read_sql_query(query_3_1, connection)
vacancies_df

cnt_vacancies
0          49197

*Количество вакансий в базе - 49197*

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [5]:
query_3_2 = f'''
SELECT COUNT(id) cnt_employers
    FROM public.employers            
'''
employers_df = pd.read_sql_query(query_3_2, connection)
employers_df

cnt_employers
0          23501

*Количество работодателей в базе - 23501*

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [6]:
query_3_3 = f'''
SELECT COUNT(id) cnt_areas
    FROM public.areas              
'''
areas_df = pd.read_sql_query(query_3_3, connection)
areas_df

cnt_areas
0       1362

*Количество регионов в базе - 1362*

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [7]:
query_3_4= f'''
SELECT COUNT(id) cnt_industries
    FROM public.industries              
'''
industries_df = pd.read_sql_query(query_3_4, connection)
industries_df

cnt_industries
0             294

*Количество сфер деятельности в базе - 294*

***

### Выводы из предварительного анализа

*Предварительный анализ показал, что база данных содержит 49197 уникальных вакансий от 23501 уникальных работодателей.*
*Вышеперечисленные вакансии представлены в 294-х различных сферах деятельности.*
* *В среднем каждый работодатель размещает 2 вакансии.*
* *В каждом регионе в среднем 36 вакансий.*
* *В среднем по каждой сфере деятельности открыто около 170 вакансий.*
* *Количество регионов превышает количество городов России, что говорит о том, что на hh представлены города других стран, либо есть ошибки в данных (различное написания одного и того же города)**

*Очевидно, требуется более детальный анализ данных.*

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [8]:
query_4_1 = f'''
SELECT 
    a.name area,
    COUNT(v.id) cnt
FROM public.vacancies v
JOIN public.areas a ON a.id = v.area_id
GROUP BY a.name
ORDER BY 2 DESC
'''
vac_in_area_df = pd.read_sql_query(query_4_1, connection)
vac_in_area_df.head(10)

area   cnt
0           Москва  5333
1  Санкт-Петербург  2851
2            Минск  2112
3      Новосибирск  2006
4           Алматы  1892
5     Екатеринбург  1698
6  Нижний Новгород  1670
7           Казань  1415
8        Краснодар  1301
9           Самара  1144

*В пятёрку лидеров по количеству вакансий входят Москва, Санкт-Петербург, Минск, Новосибирск, Алматы*

2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [9]:
query_4_2 = f'''
SELECT 
    COUNT(v.id)
FROM public.vacancies v
WHERE 
   salary_from IS NOT NULL 
   OR salary_to IS NOT NULL
'''
salary_df = pd.read_sql_query(query_4_2, connection)
salary_df.head(10)

count
0  24073

*Количество вакансий, у которых заполнено хотя бы одно из двух полей с зарплатой - 24073, что составляет меньше половины от всех вакансий*

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [10]:
query_4_3 = f'''
SELECT 
    AVG(salary_from) avg_salary_from,
    AVG(salary_to) avg_salary_to
FROM public.vacancies v
'''
salary_df = round(pd.read_sql_query(query_4_3, connection), 0)
salary_df

avg_salary_from  avg_salary_to
0          71065.0       110537.0

*Среднее значение нижней границы зарплатной вилки составляет 71065, верхней - 110537*

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [11]:
query_4_4 = f'''
SELECT 
    v.schedule,
    v.employment,
    COUNT(v.id) cnt
FROM public.vacancies v
GROUP BY 1, 2
ORDER BY cnt DESC
'''
salary_df = pd.read_sql_query(query_4_4, connection)
salary_df

schedule           employment    cnt
0        Полный день     Полная занятость  35367
1   Удаленная работа     Полная занятость   7802
2      Гибкий график     Полная занятость   1593
3   Удаленная работа  Частичная занятость   1312
4     Сменный график     Полная занятость    940
5        Полный день           Стажировка    569
6     Вахтовый метод     Полная занятость    367
7        Полный день  Частичная занятость    347
8      Гибкий график  Частичная занятость    312
9        Полный день     Проектная работа    141
10  Удаленная работа     Проектная работа    133
11     Гибкий график           Стажировка    116
12    Сменный график  Частичная занятость    101
13  Удаленная работа           Стажировка     64
14     Гибкий график     Проектная работа     18
15    Сменный график           Стажировка     12
16    Вахтовый метод     Проектная работа      2
17    Сменный график     Проектная работа      1

*На втором месте по популярности для сочетания типа рабочего графика и типа трудоустройства находится пара "Удаленная работа - Полная занятость", с количеством вакансий 7802*

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [12]:
query_4_5 = f'''
SELECT 
    v.experience,
    COUNT(v.id) cnt   
FROM public.vacancies v
GROUP BY 1
ORDER BY cnt
'''
salary_df = pd.read_sql_query(query_4_5, connection)
salary_df

experience    cnt
0         Более 6 лет   1337
1           Нет опыта   7197
2       От 3 до 6 лет  14511
3  От 1 года до 3 лет  26152

*Наибольшее количество вакансий представлено для опыта работы от 1 года до 3 лет: 26152 шт*  
*Наименьшее количество вакансий - для специалистов с опытом работы более 6 лет: 1337 шт*

***

### Выводы по детальному анализу вакансий

* *Прослеживается прямая зависимость количества вакансий в городах от количества жителей*
* *Примерно в половине вакансий не указана заработная плата*
* *Подавляющее большинство вакансий для полного рабочего дня и полной занятости, но также есть и другие варианты рабочего графика и типа трудоустройства*
* *Более половины вакансий в базе - с требуемым опытом работы от 1 года до 3 лет*


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [13]:
query_5_1 = f'''
SELECT 
    e.name employer, 
    count(v.id) vacancies_count
FROM public.vacancies v 
JOIN public.employers e ON v.employer_id = e.id
GROUP BY employer
ORDER BY vacancies_count desc
LIMIT 5
'''
df = pd.read_sql_query(query_5_1, connection)
display(df)

employer  vacancies_count
0         Яндекс             1933
1     Ростелеком              491
2       Тинькофф              444
3           СБЕР              428
4  Газпром нефть              331

*По количеству вакансий на первом месте с большим отрывом находится Яндекс, на пятом - Газпромнефть*

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [14]:
query_5_2 = f'''
SELECT
    a.name,
    COUNT(e.id) emp_cnt,
    COUNT(v.id) vac_cnt
FROM public.areas a
LEFT JOIN public.employers e ON e.area = a.id
LEFT JOIN public.vacancies v ON v.area_id = a.id AND v.employer_id = e.id
GROUP BY a.name
ORDER BY
    vac_cnt,
    emp_cnt DESC
'''
df = pd.read_sql_query(query_5_2, connection)
display(df)

name  emp_cnt  vac_cnt
0                 Россия      410        0
1              Казахстан      207        0
2     Московская область       75        0
3         Великобритания       23        0
4     Краснодарский край       19        0
...                  ...      ...      ...
1357        Екатеринбург     1166      960
1358              Алматы     1276     1069
1359     Санкт-Петербург     3069     1811
1360               Минск     2228     1858
1361              Москва     8554     4567

[1362 rows x 3 columns]

*Среди регионов, в которых нет вакансий, по количеству работодателей лидирует регион "Россия": 410 работодателей*

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [15]:
query_5_3 = f'''
SELECT
    e.name,
    COUNT(DISTINCT v.area_id) areas_cnt
FROM public.employers e
JOIN public.areas a ON e.area = a.id
JOIN public.vacancies v ON e.id = v.employer_id
WHERE v.employer_id IS NOT NULL and v.area_id IS NOT NULL 
GROUP BY e.name
ORDER BY areas_cnt DESC
'''
df = pd.read_sql_query(query_5_3, connection)
display(df)

name  areas_cnt
0                      Яндекс        181
1                  Ростелеком        152
2                  Спецремонт        116
3      Поляков Денис Иванович         88
4                   ООО ЕФИН          71
...                       ...        ...
14761                  UniSol          1
14762            UNISTORY LLC          1
14763                   UNIT6          1
14764     United Distribution          1
14765              UNITEL LLC          1

[14766 rows x 2 columns]

*Компания "Яндекс" публикует свои вакансии в наибольшем количестве регионов*

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [16]:
query_5_4 = f'''
SELECT
    COUNT(e.name)
FROM public.employers e
LEFT JOIN public.employers_industries ei ON e.id = ei.employer_id
WHERE ei.industry_id is NULL
'''
df = pd.read_sql_query(query_5_4, connection)
display(df)

count
0   8419

*У 8419 работодателей не указана сфера деятельности*

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [17]:
query_5_5 = f'''
SELECT
    e.name
FROM public.employers e
LEFT JOIN public.employers_industries ei ON e.id = ei.employer_id
GROUP BY e.name
HAVING COUNT(ei.industry_id) = 4
ORDER BY e.name
OFFSET 2 LIMIT 1 
'''
df = pd.read_sql_query(query_5_5, connection)
display(df)

name
0  2ГИС

*Название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности - 2ГИС.*

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [18]:
query_5_6 = f'''
SELECT
    COUNT(e.name)
FROM public.employers e
JOIN public.employers_industries ei ON e.id = ei.employer_id
JOIN public.industries i ON ei.industry_id = i.id
WHERE i.name = 'Разработка программного обеспечения'
'''
df = pd.read_sql_query(query_5_6, connection)
display(df)

count
0   3553

*У 3553 компаний в качестве сферы деятельности указана Разработка программного обеспечения*

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [20]:
# код для получения списка городов-милионников
import requests # Импортируем библиотеку requests
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup
import re # Импортируем библиотеку регулярных выражений

url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России' 
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
cities = page.find('table',  class_='standard sortable') # Находим таблицу с городами-миллиониками на сайте
cities = cities.find_all('tr')

# Составим список из имен городов-миллиоников
cities_list = []
for city in cities[1:]: # убираем заголовок таблицы
    cities_list.append(city.find('td').find_next_sibling('td').find('a').get('title', 'no title'))
cities_tup = tuple(cities_list) # преобразуем в кортеж для дальнейшей работы

***

In [21]:
# текст запроса
query_5_7 = f'''
SELECT 
    a.name,
    COUNT(v.id) vac_cnt
FROM public.areas a
JOIN public.vacancies v ON a.id = v.area_id
JOIN public.employers e on e.id = v.employer_id
WHERE e.name = 'Яндекс' AND a.name IN {cities_tup}
GROUP BY a.name

UNION ALL

SELECT
'Total', COUNT(v.id)
  FROM public.vacancies v 
  JOIN public.employers e ON e.id=v.employer_id
  JOIN public.areas a ON a.id=v.area_id
  WHERE e.name = 'Яндекс' AND a.name IN {cities_tup}
GROUP BY e.name
ORDER BY vac_cnt 
'''
df = pd.read_sql_query(query_5_7, connection)
display(df)
print("Количество строк в таблице -", df.shape[0])
print("Значение строки `Total' -", df[df['name'] == 'Total']['vac_cnt'].iloc[0])

name  vac_cnt
0              Омск       21
1         Челябинск       22
2        Красноярск       23
3         Волгоград       24
4             Пермь       25
5            Казань       25
6    Ростов-на-Дону       25
7               Уфа       26
8            Самара       26
9         Краснодар       30
10          Воронеж       32
11      Новосибирск       35
12  Нижний Новгород       36
13     Екатеринбург       39
14  Санкт-Петербург       42
15           Москва       54
16            Total      485

Количество строк в таблице - 17
Значение строки `Total' - 485


*Компания "Яндекс" предоставляет вакансии во всех городах-милионниках. Лидирует по количеству вакансий Москва. Всего компания "Яндекс" предлагает 485 вакансий в городах-милионниках.*


***

### Выводы по анализу работодателей

* *Топ 5 работодателей по количеству вакансий на hh.ru - Яндекс, Ростелеком, Тинькофф, СБЕР, Газпром нефть. На них суммарно приходится 3627 вакансии или 7.4% от общего числа вакансий*
* *8419 компаний (около 35%) не указали сферу деятельности*
* *3553 работодателя указали сферу деятельности "Разработка ПО"*
* *В некоторых регионах нет вакансий, не смотря на большое количество работодателей в них*

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [22]:
query_6_1 = f'''
SELECT
    COUNT(v.id)
FROM public.vacancies v
WHERE lower(v.name) LIKE '%data%' OR LOWER(v.name) LIKE '%данн%'
'''
df = pd.read_sql_query(query_6_1, connection)
display(df)

count
0   1771

*Количество вакансий, которые имеют отношение к данным - 1771*

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [23]:
query_6_2 = f'''
SELECT
    COUNT(v.id)
FROM public.vacancies v
WHERE (LOWER(v.name) LIKE '%data scientist%' 
OR LOWER(v.name) LIKE '%data science%'
OR LOWER(v.name) LIKE '%исследователь данных%'
OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
OR LOWER(v.name) LIKE '%machine learning%'
OR LOWER(v.name) LIKE '%машинн%обучен%')
AND (LOWER(v.name) LIKE '%junior%'
    OR LOWER(v.experience) LIKE '%нет опыта%'
    OR LOWER(v.employment) LIKE '%стажировка%')
'''
df = pd.read_sql_query(query_6_2, connection)
display(df)

count
0     51

*Для начинающего дата-сайентиста в базе есть 51 вакансия*

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [24]:
query_6_3 = f'''
SELECT
    COUNT(v.id)
FROM public.vacancies v
WHERE (LOWER(v.name) LIKE '%data scientist%' 
OR LOWER(v.name) LIKE '%data science%'
OR LOWER(v.name) LIKE '%исследователь данных%'
OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
OR LOWER(v.name) LIKE '%machine learning%'
OR LOWER(v.name) LIKE '%машинн%обучен%')
AND (UPPER(key_skills) LIKE '%SQL%' OR LOWER(key_skills) LIKE '%postgres%')
'''
df = pd.read_sql_query(query_6_3, connection)
display(df)

count
0    201

*В базе представлена 201 вакансия для дата-сайентиста, где в качестве ключевого навыка указан SQL или postgres*

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [25]:
query_6_4 = f'''
SELECT
    COUNT(v.id)
FROM public.vacancies v
WHERE (LOWER(v.name) LIKE '%data scientist%' 
OR LOWER(v.name) LIKE '%data science%'
OR LOWER(v.name) LIKE '%исследователь данных%'
OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
OR LOWER(v.name) LIKE '%machine learning%'
OR LOWER(v.name) LIKE '%машинн%обучен%')
AND (LOWER(key_skills) LIKE '%python%')
'''
df = pd.read_sql_query(query_6_4, connection)
display(df)

count
0    351

*В базе представлена 351 вакансия для дата-сайентиста, где в качестве ключевого навыка указан Python*

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [26]:
query_6_5 = f'''
SELECT
    ROUND(AVG(array_length(regexp_split_to_array(key_skills, '\t'), 1)), 2) avg_skills_count 
FROM public.vacancies v
WHERE (LOWER(v.name) LIKE '%data scientist%' 
OR LOWER(v.name) LIKE '%data science%'
OR LOWER(v.name) LIKE '%исследователь данных%'
OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
OR LOWER(v.name) LIKE '%machine learning%'
OR LOWER(v.name) LIKE '%машинн%обучен%')
'''
df = pd.read_sql_query(query_6_5, connection)
display(df)

avg_skills_count
0              6.41

*В вакансиях для DS в среднем указывается 6,41 ключевых навыков*

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [27]:
query_6_6 = f'''
SELECT 
v.experience,
ROUND(AVG(COALESCE((v.salary_from + v.salary_to)/2, v.salary_from, v.salary_to, 0)), 0) salary
FROM public.vacancies v
WHERE ( 
LOWER(v.name) LIKE '%data scientist%' 
OR LOWER(v.name) LIKE '%data science%'
OR LOWER(v.name) LIKE '%исследователь данных%'
OR (v.name LIKE '%ML%' AND lower(v.name) NOT LIKE '%html%')
OR LOWER(v.name) LIKE '%machine learning%'
OR LOWER(v.name) LIKE '%машинн%обучен%'
  )
AND (v.salary_from IS NOT NULL
OR v.salary_to IS NOT NULL)
GROUP BY v.experience
'''
df = pd.read_sql_query(query_6_6, connection)
display(df)

experience    salary
0           Нет опыта   74643.0
1  От 1 года до 3 лет  139675.0
2       От 3 до 6 лет  243115.0

***

*Средняя зарплата дата-сайентиста с опытом работы от 3 до 6 лет - 243115 рублей*

### Выводы по предметному анализу
* *В HeadHunter представлена 1771 вакансия, имеющая отношение к данным. Из них только 51 вакансия подойдет для начинающих дата-сайентистов.*
* *Среднее количество ключевых навыков для дата-сайентистов - 6,41. Для понимания, много это или мало, требуется дополнительное исследование.*
* *Заработная плата дата-сайентистов стремительно растет с увеличением опыта работы.*

# Общий вывод по проекту

*Для начала узнаем, сколько всего вакансий для дата-сайентистов любого уровня представлено в базе*

In [28]:
query_7_1 = f'''
SELECT
    COUNT(v.id)
FROM public.vacancies v
WHERE (LOWER(v.name) LIKE '%data scientist%' 
OR LOWER(v.name) LIKE '%data science%'
OR LOWER(v.name) LIKE '%исследователь данных%'
OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
OR LOWER(v.name) LIKE '%machine learning%'
OR LOWER(v.name) LIKE '%машинн%обучен%')
'''
df = pd.read_sql_query(query_7_1, connection)
display(df)

count
0    480

*Из 1771 вакансии, относящейся к данным, только 480 - для дата-сайентистов, что составляет около 1% от всех вакансий в базе.*  
*10% от всех вакансий для дата-сайентистов - для специалистов начального уровня.*  
*Знание SQL или Postgres требуется в 42% вакансий для DS, знание Python - в 73%.*


*Узнаем количество вакансий для дата-сайентистов по регионам (ТОП-5)*

In [29]:
query_7_2 = f'''
SELECT
    a.name,
    COUNT(v.name) 
FROM public.vacancies v
JOIN public.areas a ON v.area_id = a.id
WHERE (LOWER(v.name) LIKE '%data scientist%' 
OR LOWER(v.name) LIKE '%data science%'
OR LOWER(v.name) LIKE '%исследователь данных%'
OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
OR LOWER(v.name) LIKE '%machine learning%'
OR LOWER(v.name) LIKE '%машинн%обучен%')
GROUP BY a.name
ORDER BY 2 DESC
LIMIT 5
'''
df = pd.read_sql_query(query_7_2, connection)
display(df)

name  count
0           Москва    217
1  Санкт-Петербург     64
2      Новосибирск     23
3  Нижний Новгород     20
4           Казань     16

*Как и ожидалось, Москва лидирует по этому показателю с большим отрывом.*

*Теперь посмотрим ТОП-10 компаний по количеству вакансий для DS*

In [30]:
query_7_3 = f'''
SELECT
    e.name,
    COUNT(v.name)
FROM public.employers e
JOIN public.vacancies v ON e.id = v.employer_id
WHERE (LOWER(v.name) LIKE '%data scientist%' 
OR LOWER(v.name) LIKE '%data science%'
OR LOWER(v.name) LIKE '%исследователь данных%'
OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
OR LOWER(v.name) LIKE '%machine learning%'
OR LOWER(v.name) LIKE '%машинн%обучен%')
GROUP BY e.name
ORDER BY 2 DESC
LIMIT 10
'''
df = pd.read_sql_query(query_7_3, connection)
display(df)

name  count
0                   СБЕР     37
1        Bell Integrator     25
2         Банк ВТБ (ПАО)     18
3                     VK     15
4  Positive Technologies     11
5                 Яндекс      9
6         EvenBet Gaming      9
7                МегаФон      8
8               Andersen      7
9                   Ozon      6

*Почти 8% всех вакансий для DS от СБЕРа, а вот Яндекс разместил всего 9 вакансий для DS.*

*Сравним среднюю ЗП начинающего дата-сайентиста со средней ЗП по всем вакансиям в базе. Среднюю ЗП начинающего дата-сайентиста мы уже знаем, найдем среднюю ЗП по всем вакансиям*

In [31]:
query_7_4 = f'''
SELECT 
v.experience,
ROUND(AVG(COALESCE((v.salary_from + v.salary_to)/2, v.salary_from, v.salary_to, 0)), 0) salary
FROM public.vacancies v
WHERE (v.salary_from IS NOT NULL
OR v.salary_to IS NOT NULL)
GROUP BY v.experience
'''
df = pd.read_sql_query(query_7_4, connection)
display(df)

experience    salary
0         Более 6 лет  166256.0
1       От 3 до 6 лет  133211.0
2           Нет опыта   40472.0
3  От 1 года до 3 лет   76541.0

*Средняя заработная плата начинающего дата-сайентиста почти в 2 раза выше, чем в среднем по всем вакансиям. Похоже, курс по DS оказался правильным выбором.*

*Узнаем сколько вакансий для дата-сайентистов предполагают удаленную работу.*

In [32]:
query_7_5 = f'''
SELECT 
COUNT(v.name)
FROM public.vacancies v
WHERE (LOWER(v.name) LIKE '%data scientist%' 
OR LOWER(v.name) LIKE '%data science%'
OR LOWER(v.name) LIKE '%исследователь данных%'
OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%')
OR LOWER(v.name) LIKE '%machine learning%'
OR LOWER(v.name) LIKE '%машинн%обучен%')
AND LOWER(v.schedule) LIKE '%удал%'
'''
df = pd.read_sql_query(query_7_5, connection)
display(df)

count
0    115

*115 вакансий из 480 предполагают удаленную работу, почти четверть*

*Было бы интересно понаблюдать за проанализированными данными в динамике, чтобы посмотреть как меняется рынок труда в целом и для дата-сайентистов в частности. Что происходит с количеством вакансий для DS - увеличивается или уменьшается? Как меняется зарплата, как средняя, так и в зависимоти от опыта/региона/ графика работы? Появляются ли новые ключевые наквыки?*

In [33]:
connection.close()